In [1]:
# Imports
import subprocess
import json
import os
import dotenv

from constants import *
from bit import PrivateKeyTestnet
from eth_account import Account
from web3 import Web3

In [2]:
dotenv.load_dotenv()

w3=Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [3]:
# Mnemonic variable for cryptowallet private key stored in .env file
mnemonic = os.getenv("mnemonic", "mnemonic")



# Establish variable for derive command 
#command = f'./derive -g --mnemonic=mnemonmic --cols=path,address,privkey,pubkey --numderive=3 --format=json --coin={coin}'
# Ended up hard coding 'command' variable as parameter in subprocess.Popen() below

In [4]:
# Create function to derive wallets
def derive_wallets(coin=BTC):

    p = subprocess.Popen(f'./derive -g --mnemonic=mnemonmic --cols=path,address,privkey,pubkey --numderive=3 --format=json --coin={coin}', 
                         stdout=subprocess.PIPE, 
                         shell=True)
    
    output, err = p.communicate()
    p_status = p.wait()

    keys = json.loads(output)
    
    # Create dictionary to assign 4 keys to each currency created in constants.py
    #coins = {
    #         BTC: [keys[0], keys[1], keys[2], keys[3]],
    #         ETH: [keys[4], keys[5], keys[6], keys[7]],
    #         BTCTEST: [keys[8], keys[9], keys[10], keys[11]]
    #        }
    
    #print(coins)
    
    
    return(keys)
    
    # Test query
    #print(coins[ETH][2]['privkey'])

    
#derive_wallets()
#[0]['privkey']

coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST)
}

# Confirm coin object
coins


{'eth': [{'path': "m/44'/60'/0'/0/0",
   'address': '0xaea6b209847D73b276520f21594287Ab36eaed66',
   'privkey': '0xc0f298143e4561e38c0a37211ec879564e33cb9e86e2524719643c2dc7f6c2f4',
   'pubkey': '024c5d0c28cc3b582901957b3ead95ec8e2c008f350f5dee8721063503456c3480'},
  {'path': "m/44'/60'/0'/0/1",
   'address': '0xdABa555B2925D8EaB3fDc422750c6De47FEdee60',
   'privkey': '0x59856c0125c7e5ecfa307b38859a935ad2840c3fa7c3642ab61187615e482d3d',
   'pubkey': '02ceccb3e37aa87555178dfa93e3f07b18a3c291f3d67c4c8528ceadb6f4beb596'},
  {'path': "m/44'/60'/0'/0/2",
   'address': '0xB07014b452c15f3BF482A505911CCc1f45f2Fe33',
   'privkey': '0x9701386cd6815833eb573e0f18afe2f9cdc028791113b1f2cc3edb7bedfabc09',
   'pubkey': '0358912d3173212b709b64be6f23fc863473f74742cea136f5755b7eb9a6cecfa6'}],
 'btc-test': [{'path': "m/44'/1'/0'/0/0",
   'address': 'mpr71pzE5nz74p9UG3eCA5T72SNTpsx7Ry',
   'privkey': 'cTfaomzWuQJTa7dNJ5cpfgRH6pEacsmEws2C1NDj6334kmMRVEQM',
   'pubkey': '03916643da271d4758d40d780c640d833d63f

In [5]:
# Establish private key variable
#private_key = derive_wallets()[ETH][0]['privkey']
#private_key = b'L2XCeB5qiTDu5BiCUNr3RX7khemTGqZmFEzW2rDUN8PTAf1kuuj5'
#private_key = derive_wallets()[0]['privkey']

# ETH key
#private_key = coins[ETH][0]['privkey']

# BTCTESTNET key
private_key = coins[BTCTEST][0]['privkey']

#Account.privateKeyToAccount(private_key)

# Convert string variable (above) to hex
#an_integer = int(private_key, 16)
private_key

'cTfaomzWuQJTa7dNJ5cpfgRH6pEacsmEws2C1NDj6334kmMRVEQM'

In [6]:
# Create function to convert private key string to an account object usable in bit or web3
def privkey_to_account(coin, privkey):
    
    if coin == ETH:
        return Account.privateKeyToAccount(privkey)
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet(privkey)


# Test function    
#print(privkey_to_account(ETH, private_key))
print(privkey_to_account(BTCTEST, private_key))

a = b'asfd908432nklD'
# binary string - hashes are usually in this form



<PrivateKeyTestnet: mpr71pzE5nz74p9UG3eCA5T72SNTpsx7Ry>


In [7]:
# Create function to create raw, unsigned transaction
def create_tx(coin, account, to, amount):
    
    if coin == ETH:
        
        gasEstimate = w3.eth.estimateGas({"from": account.address, 
                                      "to": to, 
                                      "value": amount})
        
        return {"from": account.address, 
                "to": to, 
                "value": amount, 
                "gasPrice": w3.eth.gasPrice, 
                "gas": gasEstimate, 
                "nonce": w3.eth.getTransactionCount(account.address)}
    
    elif coin == BTCTEST:
        
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [8]:
# Create function to sign transaction and send to designated network
def send_tx(coin, account, to, amount):
    
    raw_tx = create_tx(coin, account, to, amount)
    
    if coin == ETH:
        
        signed_tx = account.signed_transaction(rax_tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result
        
    
    elif coin == BTCTEST:
        
        signed_tx = PrivateKeyTestnet.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed_tx)
        # Probably need a line to convert raw transaction to signed transaction

In [9]:
# Create BTCTEST recipient address object
recipient_address = privkey_to_account(BTCTEST, coins[BTCTEST][1]['privkey'])
recipient_address = coins[BTCTEST][1]['privkey']
recipient_address

'cSn1D7XronX9xP7CqsxchNmUgZzJCMm3vUGiHZGPtzeFuZ4GNRQX'

In [11]:
# Create and send ETH transaction

send_tx(BTCTEST, privkey_to_account(BTCTEST, private_key), recipient_address, 0.0001)

ValueError: b'\xef' does not correspond to a mainnet nor testnet address.